In [18]:
# imports
import pandas as pd
import pandas_profiling
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import warnings
import statistics as st                                                 # for statistical analysis
from sklearn.preprocessing import StandardScaler, normalize             # data scaling and normalization
from sklearn.cluster import KMeans
import plotly.express as px
warnings.filterwarnings("ignore")

In [2]:
# setting up several paths 
sys.path.append(".")
sys.path.append("..")
sys.path.insert(1, '../scripts')
from dataCleaner import *
from dataVisualizer import *

In [3]:
# read data
df = pd.read_csv('../data/Week1_challenge_data_source_filled_2.csv.bz2')
df

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,total_gaming,total_youtube,total_netflix,total_social,fixed_dur,fixed_activity_dur_dl,fixed_activity_dur_ul,fixed_total_ul,total_data,decile
0,1.311450e+19,2019-04-04 12:01:00,770.0,2019-04-25 14:35:00,662.0,86399.0,2.082010e+14,3.366496e+10,3.552120e+13,9.16457E+15,...,2905912.0,2905912.0,2905912.0,2905912.0,86399.0,37624.0,38787.0,36749741.0,345629377.0,"(72290.0, 86399.0]"
1,1.311450e+19,2019-04-09 13:04:00,235.0,2019-04-25 08:15:00,606.0,86399.0,2.082020e+14,3.368185e+10,3.579400e+13,L77566A,...,4414096.0,4414096.0,4414096.0,4414096.0,86399.0,168.0,3560.0,53800391.0,707185356.0,"(72290.0, 86399.0]"
2,1.311450e+19,2019-04-09 17:42:00,1.0,2019-04-25 11:58:00,652.0,86399.0,2.082000e+14,3.376063e+10,3.528150e+13,D42335A,...,10229119.0,10229119.0,10229119.0,10229119.0,86399.0,0.0,0.0,27883638.0,307690973.0,"(72290.0, 86399.0]"
3,1.311450e+19,2019-04-10 00:31:00,486.0,2019-04-25 07:36:00,171.0,86399.0,2.082010e+14,3.375034e+10,3.535660e+13,T21824A,...,11811761.0,11811761.0,11811761.0,11811761.0,86399.0,3330.0,37882.0,43324218.0,889352748.0,"(72290.0, 86399.0]"
4,1.311450e+19,2019-04-12 20:10:00,565.0,2019-04-25 10:40:00,954.0,86399.0,2.082010e+14,3.369980e+10,3.540700e+13,D88865A,...,7748843.0,7748843.0,7748843.0,7748843.0,86399.0,0.0,0.0,38542814.0,607681403.0,"(72290.0, 86399.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,1.304240e+19,2019-04-29 07:28:00,615.0,2019-04-30 00:01:00,407.0,59587.0,2.082010e+14,3.366865e+10,3.533250e+13,T85721A,...,1265634.0,1265634.0,1265634.0,1265634.0,59587.0,3380381.0,2355730.0,53745392.0,872761860.0,"(45959.8, 72290.0]"
149996,7.277830e+18,2019-04-29 07:28:00,451.0,2019-04-30 06:02:00,214.0,81230.0,2.082020e+14,3.365069e+10,3.548310e+13,D20434A,...,12784914.0,12784914.0,12784914.0,12784914.0,81230.0,42376.0,41915.0,57628851.0,631804110.0,"(72290.0, 86399.0]"
149997,7.349880e+18,2019-04-29 07:28:00,483.0,2019-04-30 10:41:00,187.0,97970.0,2.082020e+14,3.366345e+10,3.566050e+13,D10223C,...,4415361.0,4415361.0,4415361.0,4415361.0,97970.0,17264.0,16759.0,39135081.0,705783925.0,"(86400.0, 101468.0]"
149998,1.311450e+19,2019-04-29 07:28:00,283.0,2019-04-30 10:46:00,810.0,98249.0,2.082020e+14,3.362189e+10,3.572120e+13,T51102A,...,6585469.0,6585469.0,6585469.0,6585469.0,98249.0,25003.0,28640.0,34912224.0,627698629.0,"(86400.0, 101468.0]"


In [4]:
# create a df cleaner instance
cleaner = dataCleaner(df)
visualizer = dataVisualizer()

Data cleaner in action.
Data visualizer in action.


# User engagement analysis
* Tracking the user engagement based on session frequency, duration of session, and total traffic (UL+DL) on the network

#### Aggregating the above metrics per customer id (MSISDN) and report the top 10 customers per engagement metric

In [5]:
user_engagement_df = df[['MSISDN/Number', 'Bearer Id', 'Dur. (ms)', 'total_data']]

user_engagement_df = user_engagement_df.groupby(
    'MSISDN/Number').agg({'Bearer Id': 'count', 'Dur. (ms)': 'sum', 'total_data': 'sum'})
user_engagement_df = user_engagement_df.rename(
    columns={'Bearer Id': 'XDR Sessions'})
user_engagement_df

,XDR Sessions,Dur. (ms),total_data
MSISDN/Number,,,
3.360100e+10,1,116720.0,8.786906e+08
3.360100e+10,1,181230.0,1.568596e+08
3.360100e+10,1,134969.0,5.959665e+08
3.360101e+10,1,49878.0,4.223207e+08
3.360101e+10,2,37104.0,1.457411e+09
...,...,...,...
3.379000e+10,1,8810.0,7.146416e+08
3.379000e+10,1,140988.0,4.803073e+08
3.197020e+12,1,86399.0,2.321240e+08


In [6]:
user_engagement_df['XDR Sessions'].value_counts()

1       77730
2       21452
3        4422
4        1950
5         938
6         208
7          75
8          34
9          17
10         12
12          5
11          5
17          2
15          2
18          1
16          1
1066        1
13          1
Name: XDR Sessions, dtype: int64

In [7]:
#  top 10 customers per engagement metric - XDR sessions
user_engagement_df.nlargest(10, 'XDR Sessions')

,XDR Sessions,Dur. (ms),total_data
MSISDN/Number,,,
3.366371e+10,1066,63940818.0,5.302862e+11
3.362632e+10,18,1785419.0,7.951348e+09
3.361489e+10,17,1678397.0,8.826326e+09
3.362578e+10,17,1599052.0,8.474657e+09
3.365973e+10,16,1905478.0,7.680848e+09
3.367588e+10,15,1745131.0,7.891111e+09
3.376054e+10,15,1651084.0,8.495067e+09
3.366716e+10,13,1209703.0,5.618394e+09
3.360313e+10,12,1109736.0,4.976195e+09


* These are the top 10 users per the XDR session engagement metric

In [8]:
#  top 10 customers per engagement metric - Durations
user_engagement_df.nlargest(10, 'Dur. (ms)')

,XDR Sessions,Dur. (ms),total_data
MSISDN/Number,,,
3.366371e+10,1066,63940818.0,5.302862e+11
3.365973e+10,16,1905478.0,7.680848e+09
3.362632e+10,18,1785419.0,7.951348e+09
3.365936e+10,11,1747994.0,4.658437e+09
3.367588e+10,15,1745131.0,7.891111e+09
3.361489e+10,17,1678397.0,8.826326e+09
3.376054e+10,15,1651084.0,8.495067e+09
3.362578e+10,17,1599052.0,8.474657e+09
3.378632e+10,12,1427807.0,5.622232e+09


* These are the top 10 users per the duration of session engagement metric

In [9]:
#  top 10 customers per engagement metric - total data volume
user_engagement_df.nlargest(10, 'total_data')

,XDR Sessions,Dur. (ms),total_data
MSISDN/Number,,,
3.366371e+10,1066,63940818.0,5.302862e+11
3.361489e+10,17,1678397.0,8.826326e+09
3.376054e+10,15,1651084.0,8.495067e+09
3.362578e+10,17,1599052.0,8.474657e+09
3.362632e+10,18,1785419.0,7.951348e+09
3.367588e+10,15,1745131.0,7.891111e+09
3.365973e+10,16,1905478.0,7.680848e+09
3.366646e+10,11,1168703.0,7.308501e+09
3.376041e+10,12,1248477.0,7.112835e+09


* These are the top 10 users per the total data (UL + DL) engagement metric

#### Normalizing each engagement metric and runing a k-means (k=3) to classify customers in three groups of engagement. 

In [14]:
# Data Scaling and normalization
# Data Scaling
scaler = StandardScaler()
scaled_data = scaler.fit_transform(user_engagement_df)
print(type(scaled_data), scaled_data)

# Data Normalization
normalized_data = normalize(scaled_data)
print(type(normalized_data), normalized_data)

<class 'numpy.ndarray'> [[-0.12034413 -0.05434193  0.10890723]
 [-0.12034413  0.23939902 -0.31753898]
 [-0.12034413  0.02875338 -0.05812166]
 ...
 [-0.12034413 -0.19240609 -0.27307401]
 [-0.12034413 -0.19240609 -0.05793181]
 [-0.12034413 -0.19240609 -0.33940905]]
<class 'numpy.ndarray'> [[-0.70309945 -0.31748768  0.63628039]
 [-0.28964919  0.57619537 -0.76426582]
 [-0.88033551  0.21033535 -0.42516871]
 ...
 [-0.33893405 -0.54188746 -0.76907847]
 [-0.51380856 -0.82147667 -0.24733954]
 [-0.29475108 -0.47124776 -0.83129258]]


In [36]:
# k means clustering based on 3 clusters
k_means = KMeans(n_clusters=3, random_state=777).fit(normalized_data)
print(type(k_means.labels_), len(k_means.labels_), k_means.labels_)
print(np.unique(k_means))

<class 'numpy.ndarray'> 106856 [1 2 2 ... 1 1 1]
[KMeans(n_clusters=3, random_state=777)]


In [38]:
# insert the cluster of the data inside the dataframe
user_engagement_df.insert(0, 'cluster', k_means.labels_)
user_engagement_df.cluster.value_counts()

1    60162
0    24879
2    21815
Name: cluster, dtype: int64

We can see 3 clusters has been added

In [40]:
# the data frame with clusters added
user_engagement_df

,cluster,XDR Sessions,Dur. (ms),total_data
MSISDN/Number,,,,
3.360100e+10,1,1,116720.0,8.786906e+08
3.360100e+10,2,1,181230.0,1.568596e+08
3.360100e+10,2,1,134969.0,5.959665e+08
3.360101e+10,1,1,49878.0,4.223207e+08
3.360101e+10,1,2,37104.0,1.457411e+09
...,...,...,...,...
3.379000e+10,1,1,8810.0,7.146416e+08
3.379000e+10,2,1,140988.0,4.803073e+08
3.197020e+12,1,1,86399.0,2.321240e+08


In [41]:
# visualizing the 3 clusters in the dataframe
# for 20, 000 samples only
fig = px.scatter(user_engagement_df.sample(20000), x='total_data', y='Dur. (ms)',
                 color='cluster', size='XDR Sessions')
fig.update_traces(marker_size=8)
fig.update(layout_yaxis_range = [0, 800000])
fig.update(layout_xaxis_range = [0, 4000000000])
fig.show()

The scatter plot based on the 3 clusters
* color indicates the clusters
* size of points indicate the XDR sessions

In [43]:
# some basic description of the 3 clusters
i=0
def computeBasicAnalysisOnClusters():
    for i in range(3):
        cluster = user_engagement_df[user_engagement_df["cluster"]==i]
        print("Cluster " + (i+1) * "I")
        v = cluster[['XDR Sessions', 'Dur. (ms)', 'total_data']].describe()
        print(v)
        print("\n")
computeBasicAnalysisOnClusters()

Cluster I
       XDR Sessions     Dur. (ms)    total_data
count  24879.000000  2.487900e+04  2.487900e+04
mean       2.555529  2.496147e+05  1.294933e+09
std        6.815382  4.236554e+05  3.407945e+09
min        1.000000  3.195800e+04  2.014465e+08
25%        2.000000  1.727980e+05  8.864466e+08
50%        2.000000  2.182310e+05  1.166192e+09
75%        3.000000  3.042965e+05  1.528738e+09
max     1066.000000  6.394082e+07  5.302862e+11


Cluster II
       XDR Sessions      Dur. (ms)    total_data
count  60162.000000   60162.000000  6.016200e+04
mean       1.069030   67050.050979  5.246188e+08
std        0.253639   30377.299741  2.645020e+08
min        1.000000    7142.000000  3.324901e+07
25%        1.000000   37051.000000  3.053978e+08
50%        1.000000   84206.000000  5.242809e+08
75%        1.000000   86400.000000  7.338753e+08
max        3.000000  126213.000000  1.749061e+09


Cluster III
       XDR Sessions      Dur. (ms)    total_data
count  21815.000000   21815.000000  2.181

Insights on clusters

* cluster I: total count 24, 879
    * XDR session
        * Has a standard deviation of 6.815382
        * Relatively normal value for standard deviation
        * With a min of 1 and max of 1, 066 value
    * Duration (ms)
        * Has a standard deviation of 423,655.4
        * Relatively high value for standard deviation
        * With a min of 3,1958 and max of 6,394,082 value
    * Total data
        * Has a standard deviation of 3407945.e+03
        * Relatively high value for standard deviation
        * With a min of 2.014465e+08 and max of 5.302862e+11 value

* cluster II: total count 60, 162
    * XDR session
        * Has a standard deviation of 0.253639
        * Relatively lower value for standard deviation
        * With a min of 1 and max of 3 value
    * Duration (ms)
        * Has a standard deviation of 30377.299741
        * Relatively higher value for standard deviation
        * With a min of 7,142 and max of 126,213 value
    * Total data
        * Has a standard deviation of 2.645020e+08
        * Relatively higher value for standard deviation
        * With a min of 3.324901e+07 and max of 1.749061e+09 value

* cluster III: total count 21, 815
    * XDR session
        * Has a standard deviation of 0.114726
        * Relatively lower value for standard deviation
        * With a min of 1 and max of 2 value
    * Duration (ms)
        * Has a standard deviation of 29,080
        * Relatively lower value for standard deviation
        * With a min of 113,146 and max of 407,713 value
    * Total data
        * Has a standard deviation of 2.420226e+08
        * Relatively lower value for standard deviation
        * With a min of 3.357584e+07 and max of 9.396428e+08 value

#### Aggregating user total traffic per application and deriving the top 10 most engaged users per application

In [46]:
user_app_engagement_df = df[['MSISDN/Number', 'total_social', 'total_google',
    'total_email', 'total_youtube', 'total_netflix',
    'total_gaming', 'total_data']]
user_app_engagement_df

,MSISDN/Number,total_social,total_google,total_email,total_youtube,total_netflix,total_gaming,total_data
0,3.366496e+10,2905912.0,2905912.0,2905912.0,2905912.0,2905912.0,2905912.0,345629377.0
1,3.368185e+10,4414096.0,4414096.0,4414096.0,4414096.0,4414096.0,4414096.0,707185356.0
2,3.376063e+10,10229119.0,10229119.0,10229119.0,10229119.0,10229119.0,10229119.0,307690973.0
3,3.375034e+10,11811761.0,11811761.0,11811761.0,11811761.0,11811761.0,11811761.0,889352748.0
4,3.369980e+10,7748843.0,7748843.0,7748843.0,7748843.0,7748843.0,7748843.0,607681403.0
...,...,...,...,...,...,...,...,...
149995,3.366865e+10,1265634.0,1265634.0,1265634.0,1265634.0,1265634.0,1265634.0,872761860.0
149996,3.365069e+10,12784914.0,12784914.0,12784914.0,12784914.0,12784914.0,12784914.0,631804110.0
149997,3.366345e+10,4415361.0,4415361.0,4415361.0,4415361.0,4415361.0,4415361.0,705783925.0
149998,3.362189e+10,6585469.0,6585469.0,6585469.0,6585469.0,6585469.0,6585469.0,627698629.0


* total users and their activity engagement per application

In [48]:
# total users and their activity engagement per application
total_application_data_per_user = user_app_engagement_df.groupby('MSISDN/Number').sum()
total_application_data_per_user

,total_social,total_google,total_email,total_youtube,total_netflix,total_gaming,total_data
MSISDN/Number,,,,,,,
3.360100e+10,4389005.0,4389005.0,4389005.0,4389005.0,4389005.0,4389005.0,8.786906e+08
3.360100e+10,5334863.0,5334863.0,5334863.0,5334863.0,5334863.0,5334863.0,1.568596e+08
3.360100e+10,3443126.0,3443126.0,3443126.0,3443126.0,3443126.0,3443126.0,5.959665e+08
3.360101e+10,9678493.0,9678493.0,9678493.0,9678493.0,9678493.0,9678493.0,4.223207e+08
3.360101e+10,18499616.0,18499616.0,18499616.0,18499616.0,18499616.0,18499616.0,1.457411e+09
...,...,...,...,...,...,...,...
3.379000e+10,7531269.0,7531269.0,7531269.0,7531269.0,7531269.0,7531269.0,7.146416e+08
3.379000e+10,5429705.0,5429705.0,5429705.0,5429705.0,5429705.0,5429705.0,4.803073e+08
3.197020e+12,10438660.0,10438660.0,10438660.0,10438660.0,10438660.0,10438660.0,2.321240e+08


* Top 10 users of social media

In [49]:
# social media
social_media = total_application_data_per_user.nlargest(10, "total_social")['total_social']
social_media

MSISDN/Number
3.366371e+10    8.242894e+09
3.362632e+10    1.521919e+08
3.362578e+10    1.423079e+08
3.361489e+10    1.279738e+08
3.376054e+10    1.232231e+08
3.365973e+10    1.165163e+08
3.378632e+10    1.102545e+08
3.367588e+10    1.098605e+08
3.366716e+10    1.050327e+08
3.376127e+10    9.708999e+07
Name: total_social, dtype: float64

* Top 10 users of Goggle

In [51]:
# goggle
google = total_application_data_per_user.nlargest(10, "total_google")['total_google']
google

MSISDN/Number
3.366371e+10    8.242894e+09
3.362632e+10    1.521919e+08
3.362578e+10    1.423079e+08
3.361489e+10    1.279738e+08
3.376054e+10    1.232231e+08
3.365973e+10    1.165163e+08
3.378632e+10    1.102545e+08
3.367588e+10    1.098605e+08
3.366716e+10    1.050327e+08
3.376127e+10    9.708999e+07
Name: total_google, dtype: float64

* Top 10 users of Email

In [ ]:
# social media
social_media = total_application_data_per_user.nlargest(10, "total_social")['total_social']
social_media

* Top 10 users of Youtube

* Top 10 users of Netflix

* Top 10 users of Gaming